# Create Plots and HTML file for Stellar Parameter Overview

## Author(s): Sven Buder (SB, WG4)

This idea for this notebook is based on the documentation of SDSS-IV APOGEE DR16 ([arXiv:1912.02905](https://ui.adsabs.harvard.edu/abs/2019arXiv191202905A)):  
https://data.sdss.org/sas/dr16/apogee/spectro/aspcap/r12/l33/html/allfit.html  
We thank the people of SDSS-IV APOGEE DR16 for their documentation!

### History:
200305 SB Created

In [1]:
# Preamble for notebook 

# Compatibility with Python 3
from __future__ import (absolute_import, division, print_function)

try:
    %matplotlib inline
    %config InlineBackend.figure_format='retina'
except:
    pass

# Basic packages
import numpy as np
np.seterr(divide='ignore', invalid='ignore')
import glob
import scipy
import pandas
from astropy.table import Table, hstack, vstack
from astropy.nddata import bitmask

# Matplotlib and associated packages for plotting
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from matplotlib.transforms import Bbox,TransformedBbox
from matplotlib.image import BboxImage
from matplotlib.legend_handler import HandlerBase
from matplotlib._png import read_png
from matplotlib.backends.backend_pdf import PdfPages
from matplotlib.colors import ListedColormap
import matplotlib.colors as colors

params = {
    'font.family'        : 'sans',
    'font.size'          : 17,
    'axes.labelsize'     : 20,
    'ytick.labelsize'    : 16,
    'xtick.labelsize'    : 16,
    'legend.fontsize'    : 20,
    'text.usetex'        : True, 
    'text.latex.preamble': [r'\usepackage{upgreek}', r'\usepackage{amsmath}'],
    }   
plt.rcParams.update(params)

_parula_data = [[0.2081, 0.1663, 0.5292], 
                [0.2116238095, 0.1897809524, 0.5776761905], 
                [0.212252381, 0.2137714286, 0.6269714286], 
                [0.2081, 0.2386, 0.6770857143], 
                [0.1959047619, 0.2644571429, 0.7279], 
                [0.1707285714, 0.2919380952, 0.779247619], 
                [0.1252714286, 0.3242428571, 0.8302714286], 
                [0.0591333333, 0.3598333333, 0.8683333333], 
                [0.0116952381, 0.3875095238, 0.8819571429], 
                [0.0059571429, 0.4086142857, 0.8828428571], 
                [0.0165142857, 0.4266, 0.8786333333], 
                [0.032852381, 0.4430428571, 0.8719571429], 
                [0.0498142857, 0.4585714286, 0.8640571429], 
                [0.0629333333, 0.4736904762, 0.8554380952], 
                [0.0722666667, 0.4886666667, 0.8467], 
                [0.0779428571, 0.5039857143, 0.8383714286], 
                [0.079347619, 0.5200238095, 0.8311809524], 
                [0.0749428571, 0.5375428571, 0.8262714286], 
                [0.0640571429, 0.5569857143, 0.8239571429], 
                [0.0487714286, 0.5772238095, 0.8228285714], 
                [0.0343428571, 0.5965809524, 0.819852381], 
                [0.0265, 0.6137, 0.8135], 
                [0.0238904762, 0.6286619048, 0.8037619048], 
                [0.0230904762, 0.6417857143, 0.7912666667], 
                [0.0227714286, 0.6534857143, 0.7767571429], 
                [0.0266619048, 0.6641952381, 0.7607190476], 
                [0.0383714286, 0.6742714286, 0.743552381], 
                [0.0589714286, 0.6837571429, 0.7253857143], 
                [0.0843, 0.6928333333, 0.7061666667], 
                [0.1132952381, 0.7015, 0.6858571429], 
                [0.1452714286, 0.7097571429, 0.6646285714], 
                [0.1801333333, 0.7176571429, 0.6424333333], 
                [0.2178285714, 0.7250428571, 0.6192619048], 
                [0.2586428571, 0.7317142857, 0.5954285714], 
                [0.3021714286, 0.7376047619, 0.5711857143], 
                [0.3481666667, 0.7424333333, 0.5472666667], 
                [0.3952571429, 0.7459, 0.5244428571], 
                [0.4420095238, 0.7480809524, 0.5033142857], 
                [0.4871238095, 0.7490619048, 0.4839761905], 
                [0.5300285714, 0.7491142857, 0.4661142857], 
                [0.5708571429, 0.7485190476, 0.4493904762],
                [0.609852381, 0.7473142857, 0.4336857143], 
                [0.6473, 0.7456, 0.4188], 
                [0.6834190476, 0.7434761905, 0.4044333333], 
                [0.7184095238, 0.7411333333, 0.3904761905], 
                [0.7524857143, 0.7384, 0.3768142857], 
                [0.7858428571, 0.7355666667, 0.3632714286], 
                [0.8185047619, 0.7327333333, 0.3497904762], 
                [0.8506571429, 0.7299, 0.3360285714], 
                [0.8824333333, 0.7274333333, 0.3217], 
                [0.9139333333, 0.7257857143, 0.3062761905], 
                [0.9449571429, 0.7261142857, 0.2886428571], 
                [0.9738952381, 0.7313952381, 0.266647619], 
                [0.9937714286, 0.7454571429, 0.240347619], 
                [0.9990428571, 0.7653142857, 0.2164142857], 
                [0.9955333333, 0.7860571429, 0.196652381], 
                [0.988, 0.8066, 0.1793666667], 
                [0.9788571429, 0.8271428571, 0.1633142857], 
                [0.9697, 0.8481380952, 0.147452381], 
                [0.9625857143, 0.8705142857, 0.1309], 
                [0.9588714286, 0.8949, 0.1132428571], 
                [0.9598238095, 0.9218333333, 0.0948380952], 
                [0.9661, 0.9514428571, 0.0755333333], 
                [0.9763, 0.9831, 0.0538]]

parula = ListedColormap(_parula_data, name='parula')
parula_zero = _parula_data[0]
parula_0 = ListedColormap(_parula_data, name='parula_0')
parula_0.set_bad((1,1,1))
parula_r = ListedColormap(_parula_data[::-1], name='parula_r')

willi_blau = [0.0722666667, 0.4886666667, 0.8467]

In [2]:
def plot_density(x, y, bins=100, range=None, normed=False, weights=None, 
                 scaling=None, reduce_fn='median', smooth=0, ax=None, cmin=5, **kwargs):
    """
    Compute the bi-dimensional histogram of two data samples.

    Parameters
    ----------
    x : array_like, shape (N,)
        An array containing the x coordinates of the points to be
        histogrammed.
    y : array_like, shape (N,)
        An array containing the y coordinates of the points to be
        histogrammed.
    bins : int or array_like or [int, int] or [array, array], optional
        The bin specification:

        * If int, the number of bins for the two dimensions (nx=ny=bins).
        * If array_like, the bin edges for the two dimensions
            (x_edges=y_edges=bins).
        * If [int, int], the number of bins in each dimension
            (nx, ny = bins).
        * If [array, array], the bin edges in each dimension
            (x_edges, y_edges = bins).
        * A combination [int, array] or [array, int], where int
            is the number of bins and array is the bin edges.

    range : array_like, shape(2,2), optional
        The leftmost and rightmost edges of the bins along each dimension
        (if not specified explicitly in the `bins` parameters):
        ``[[xmin, xmax], [ymin, ymax]]``. All values outside of this range
        will be considered outliers and not tallied in the histogram.
    normed : bool, optional
        If False, returns the number of samples in each bin. If True,
        returns the bin density ``bin_count / sample_count / bin_area``.
    weights : array_like, shape(N,), optional
        An array of values ``w_i`` weighing each sample ``(x_i, y_i)``.
        Weights are normalized to 1 if `normed` is True. If `normed` is
        False, the values of the returned histogram are equal to the sum of
        the weights belonging to the samples falling into each bin.
    """
    import pylab as plt
    from scipy.ndimage import gaussian_filter

    ind = (np.isfinite(x) & np.isfinite(y))
    if False in ind:
        print("Warning: Not all values are finite.")
        w = weights
        if w is not None:
            w = w[ind]
        n, bx, by = plt.histogram2d(x[ind], y[ind], bins=bins, range=range, normed=normed,
                                    weights=w)
    else:
        n, bx, by = plt.histogram2d(x, y, bins=bins, range=range, normed=normed,
                                    weights=weights)

    if reduce_fn.lower() in ('average', 'mean', 'avg'):
        n0, _, _ = plt.histogram2d(x, y, bins=bins, range=range, normed=normed,
                                   weights=None)
        
        cmin_limit = (n0 < cmin)
        n[cmin_limit] = np.nan
        ind = n0 > 0
        n = n.astype(float)
        n[ind] /= n0[ind].astype(float)
    n[(n==0)] = np.nan
    
    defaults = dict(zorder = 2, cmap='RdYlBu_r', origin='lower', aspect='auto', rasterized=True,
                    interpolation='nearest')
    defaults.update(**kwargs)
    extent = (bx[0], bx[-1], by[0], by[-1])

    if smooth > 0:
        gaussian_filter(n, smooth, output=n)

    if ax==None:
        ax = plt.gca()
    else:
        ax=ax
        
    if scaling is None:
        scaling = 'None'
    s = scaling.lower().replace(' ', '')
    if s in (None, 'none', 'count'):
        return ax.imshow(n.T, extent=extent, **defaults)#, n
    else:
        if s in ('log', 'log(n)', 'log10', 'log(count)'):
            return ax.imshow(np.log10(n).T, extent=extent, **defaults)#, np.log10(n)
        elif s in ('log(n+1)', 'log(count+1)'):
            return ax.imshow(np.log10(n + 1).T, extent=extent, **defaults)#, np.log10(n + 1)

In [3]:
main_catalog_file = '../../../catalogs/GALAH_DR3_main.fits'
main_catalog = Table.read(main_catalog_file)

In [4]:
parameter_limits = dict()
parameter_limits['teff'] = (3000,8000)
parameter_limits['logg'] = (-0.5,5.25)
parameter_limits['fe_h'] = (-2.5,0.75)

In [5]:
def add_general_table_information():
    html_text.append(['<TABLE BORDER=2>'])
    html_text.append(['<TR><TD>N(TOTAL)<TD>'+str(len(main_catalog['sobject_id']))+'<TD>'])

In [6]:
def add_stellar_parameter_xyc_plots_per_flag(x_parameters,y_parameters,colors,bit_flag,create_new=False):

    for each_x in x_parameters:
        
        html_text.append(['<TR>'])
        
        for each_y in y_parameters:

            if each_x == each_y:
                pass
            else:

                for index_color, each_color in enumerate(colors):

                    if bit_flag=='All':
                        plottable = np.isfinite(main_catalog[each_x]) & np.isfinite(main_catalog[each_y]) & np.isfinite(main_catalog[each_color])
                    elif bit_flag==0:
                        plottable = (main_catalog['flag_sp']==0) & np.isfinite(main_catalog[each_x]) & np.isfinite(main_catalog[each_y]) & np.isfinite(main_catalog[each_color])
                    else:
                        plottable = (bitmask.bitfield_to_boolean_mask(main_catalog['flag_sp'],ignore_flags='~'+str(bit_flag))) & np.isfinite(main_catalog[each_x]) & np.isfinite(main_catalog[each_y]) & np.isfinite(main_catalog[each_color])
                    
                    if create_new:

                        print(bit_flag,each_x,each_y,each_color)

                        x_bins = np.linspace(parameter_limits[each_x][0],parameter_limits[each_x][1],200)
                        y_bins = np.linspace(parameter_limits[each_y][0],parameter_limits[each_y][1],200)
                        vmin = parameter_limits[each_color][0]
                        vmax = parameter_limits[each_color][1]

                        f, (ax1) = plt.subplots()

                        s = ax1.scatter(
                            x = np.clip(main_catalog[each_x][plottable],x_bins[0],x_bins[-1]),
                            y = np.clip(main_catalog[each_y][plottable],y_bins[0],y_bins[-1]),
                            c = np.clip(main_catalog[each_color][plottable],vmin,vmax),
                            vmin = vmin, vmax=vmax, s=0.25, rasterized=True,
                            cmap = parula,
                            zorder=1
                            )

                        plot_density(
                            x = np.clip(main_catalog[each_x][plottable],x_bins[0],x_bins[-1]),
                            y = np.clip(main_catalog[each_y][plottable],y_bins[0],y_bins[-1]),
                            bins = (x_bins,y_bins),
                            vmin = vmin,
                            vmax = vmax,
                            weights = np.clip(main_catalog[each_color][plottable],vmin,vmax),
                            reduce_fn='mean',
                            cmin = 5,
                            cmap = parula,
                            ax=ax1,
                            zorder=2,
                            rasterized=True
                            )
                        c = plt.colorbar(s,ax=ax1)

                        ax1.set_xlabel(each_x.replace('_','\_'))
                        ax1.set_ylabel(each_y.replace('_','\_'))
                        c.set_label(each_color.replace('_','\_'))

                        if each_y in ['teff','logg']:
                            ax1.invert_yaxis()
                        if each_x in ['teff','logg']:
                            ax1.invert_xaxis()

                        plt.tight_layout()
                        plt.savefig('galah_dr3_flag_'+str(bit_flag)+'_'+each_x+'_'+each_y+'_'+each_color+'.png',dpi=300,bbox_inches='tight')
                        plt.close()

                    html_text.append(['<TR><TD>N(FLAG_SP includes BITMASK '+str(bit_flag)+')<TD>'+str(len(main_catalog['sobject_id'][plottable]))+'<TD><a href=galah_dr3_flag_'+str(bit_flag)+'_'+each_x+'_'+each_y+'_'+each_color+'.png><IMG src=galah_dr3_flag_'+str(bit_flag)+'_'+each_x+'_'+each_y+'_'+each_color+'.png width=750></a>'])


In [7]:
html_text = [
    ['<HTML><BODY>'],
    ['<a href='+main_catalog_file+'> '+str(main_catalog_file)+' </a><br>'],
]

In [8]:
add_general_table_information()

In [9]:
add_stellar_parameter_xyc_plots_per_flag(
        x_parameters = ['teff'],
        y_parameters = ['logg'],
        colors = ['fe_h'],
        bit_flag='All',
        create_new=False
    )

In [10]:
add_stellar_parameter_xyc_plots_per_flag(
        x_parameters = ['teff'],
        y_parameters = ['logg'],
        colors = ['fe_h'],
        bit_flag=0,
        create_new=False
    )

In [11]:
for each_bit_flag in 2**np.arange(0,11):
    add_stellar_parameter_xyc_plots_per_flag(
        x_parameters = ['teff'],
        y_parameters = ['logg'],
        colors = ['fe_h'],
        bit_flag=each_bit_flag,
        create_new=False
    )
html_text.append(['</TABLE>'])

In [12]:
html_text.append(['</BODY></HTML>'])
np.savetxt('flag_overview.html',np.array(html_text),fmt='%s')